In [7]:
#Import data from mongoDB
import pandas as pd
import pymongo
client = pymongo.MongoClient("localhost", 27017)
db = client["BIGDATA-PROJECT"]
collection = db["MOVIES1"] 
cursor = collection.find()
data=pd.DataFrame(list(cursor))
client.close()
data=data
print(data)

                           _id                                  Name of movie  \
0     657aadbf61afffb42be104e3                       The Shawshank Redemption   
1     657aadbf61afffb42be104e4                                  The Godfather   
2     657aadbf61afffb42be104e5                                The Dark Knight   
3     657aadbf61afffb42be104e6                               Schindler's List   
4     657aadbf61afffb42be104e7  The Lord of the Rings: The Return of the King   
...                        ...                                            ...   
2424  657aadbf61afffb42be10e5b                       The Shawshank Redemption   
2425  657aadbf61afffb42be10e5c                       The Shawshank Redemption   
2426  657aadbf61afffb42be10e5d                                  The Godfather   
2427  657aadbf61afffb42be10e5e                       The Shawshank Redemption   
2428  657aadbf61afffb42be10e5f                                  The Godfather   

     Year of relase  Watcht

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split                         #LOGISCTIC REGRESSION MODEL
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


# Handling missing values in all columns
data = data.fillna('')  # Fill missing values with an empty string for all columns

# Assuming the dataset includes features 'Name of movie', 'Director', 'Star', and 'Rating_Category' as the target
features = ['Name of movie', 'Director', 'Star']
target = 'Rating_Category'  # Assuming 'Rating_Category' is the column for movie rating categories

# Convert non-string values to strings in selected columns
for feature in features:
    data[feature] = data[feature].astype(str)

# Combine selected features into a single column
data['combined_features'] = data[features].apply(lambda x: ' '.join(x), axis=1)

# Create TF-IDF matrix
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['combined_features'])

# Compute cosine similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Function to get movie suggestions based on movie title
def get_recommendations(movie_title, cosine_sim=cosine_sim, data=data):
    idx = data[data['Name of movie'].str.lower() == movie_title.lower()].index
    if len(idx):
        idx = idx[0]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:11]  # Top 10 similar movies
        movie_indices = [i[0] for i in sim_scores]
        return data['Name of movie'].iloc[movie_indices]
    else:
        return "Movie not found in the database."

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, data[target], test_size=0.2, random_state=42)

# Initialize and train a logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Generate a classification report
report = classification_report(y_test, predictions)
print("Classification Report:\n", report)

# Example: Get movie recommendations for a given movie title
suggestions = get_recommendations('Happiness')
print("\nRecommendations for 'Happiness':\n", suggestions)


NameError: name 'data' is not defined

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from imblearn.over_sampling import RandomOverSampler



# Handling missing values in all columns
data = data.fillna('')  # Fill missing values with an empty string

# Combine selected features into a single column
features = ['Name of movie', 'Director', 'Star']
data['combined_features'] = data[features].apply(lambda x: ' '.join(x), axis=1)

# Create TF-IDF matrix
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['combined_features'])

# Define the target column
target = 'Rating_Category'  # Assuming 'Movie_Rating_Category' is the column for movie rating categories

# Handling class imbalance using RandomOverSampler
oversampler = RandomOverSampler(random_state=42)
tfidf_matrix_res, target_res = oversampler.fit_resample(tfidf_matrix, data[target])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix_res, target_res, test_size=0.2, random_state=42)

# Initialize and train a Random Forest Classifier model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions using Random Forest Classifier
rf_predictions = rf_model.predict(X_test)

# Generate a classification report for Random Forest Classifier
rf_report = classification_report(y_test, rf_predictions)
print("Random Forest Classifier Report:\n", rf_report)

# Example: Get movie recommendations for a given movie title
def get_recommendations(movie_title, cosine_sim=linear_kernel(tfidf_matrix_res, tfidf_matrix_res), data=data):
    idx = data[data['Name of movie'].str.lower() == movie_title.lower()].index
    if not idx.empty:  # Check if the index is not empty
        idx = idx[0]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:11]  # Top 10 similar movies
        movie_indices = [i[0] for i in sim_scores]

        # Ensure movie indices are within the DataFrame's bounds
        valid_indices = [idx for idx in movie_indices if idx < len(data)]
        return data['Name of movie'].iloc[valid_indices]
    else:
        return "Movie not found in the database."

suggestions = get_recommendations('Happiness')
print("\nRecommendations for 'Happiness':\n", suggestions)


In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from imblearn.over_sampling import RandomOverSampler


# Handling missing values in all columns
data = data.fillna('')  # Fill missing values with an empty string for all columns

# Assuming the dataset includes features 'Name of movie', 'Director', 'Star', and 'Movie_Rating_Category' as the target
features = ['Name of movie', 'Director', 'Star']
target = 'Rating_Category'  # Assuming 'Movie_Rating_Category' is the column for movie rating categories

# Combine selected features into a single column
data['combined_features'] = data[features].apply(lambda x: ' '.join(x), axis=1)

# Create TF-IDF matrix
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['combined_features'])

# Handling class imbalance using RandomOverSampler
oversampler = RandomOverSampler(random_state=42)
tfidf_matrix_res, target_res = oversampler.fit_resample(tfidf_matrix, data[target])

# Split resampled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix_res, target_res, test_size=0.2, random_state=42)

# Initialize and train an SVM classifier
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Make predictions using SVM classifier
svm_predictions = svm_model.predict(X_test)

# Generate a classification report for SVM classifier
svm_report = classification_report(y_test, svm_predictions)
print("SVM Classifier Report:\n", svm_report)

# Example: Get movie recommendations for a given movie title
def get_recommendations(movie_title, cosine_sim=linear_kernel(tfidf_matrix_res, tfidf_matrix_res), data=data):
    idx = data[data['Name of movie'].str.lower() == movie_title.lower()].index
    if not idx.empty:  # Check if the index is not empty
        idx = idx[0]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:11]  # Top 10 similar movies

        movie_indices = [i[0] for i in sim_scores if i[0] < len(data)]  # Filter indices within DataFrame bounds

        return data['Name of movie'].iloc[movie_indices]
    else:
        return "Movie not found in the database."

suggestions = get_recommendations('Happiness')
print("\nRecommendations for 'Happiness':\n", suggestions)


SVM Classifier Report:
               precision    recall  f1-score   support

                   0.96      1.00      0.98       119
        High       1.00      1.00      1.00       111
         Low       0.76      0.78      0.77       129
    Moderate       0.77      0.71      0.74       119
   Very High       1.00      1.00      1.00       113

    accuracy                           0.89       591
   macro avg       0.90      0.90      0.90       591
weighted avg       0.89      0.89      0.89       591


Recommendations for 'Happiness':
 960         Moneyball
375          Magnolia
289     Mary and Max.
1569    Mary and Max.
993         25th Hour
Name: Name of movie, dtype: object


In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from imblearn.over_sampling import RandomOverSampler



# Handling missing values in all columns
data = data.fillna('')  # Fill missing values with an empty string

# Combine selected features into a single column
features = ['Name of movie', 'Director', 'Star']
data['combined_features'] = data[features].apply(lambda x: ' '.join(x), axis=1)

# Create TF-IDF matrix
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['combined_features'])

# Define the target column
target = 'Rating_Category'  # Assuming 'Movie_Rating_Category' is the column for movie rating categories

# Handling class imbalance using RandomOverSampler
oversampler = RandomOverSampler(random_state=42)
tfidf_matrix_res, target_res = oversampler.fit_resample(tfidf_matrix, data[target])

# Split resampled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix_res, target_res, test_size=0.2, random_state=42)

# Initialize and train a Gradient Boosting Classifier
gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
gb_model.fit(X_train, y_train)

# Make predictions using Gradient Boosting Classifier
gb_predictions = gb_model.predict(X_test)

# Generate a classification report for Gradient Boosting Classifier
gb_report = classification_report(y_test, gb_predictions)
print("Gradient Boosting Classifier Report:\n", gb_report)

# Example: Get movie recommendations for a given movie title
def get_recommendations(movie_title, cosine_sim=linear_kernel(tfidf_matrix_res, tfidf_matrix_res), data=data):
    idx = data[data['Name of movie'].str.lower() == movie_title.lower()].index
    if not idx.empty:
        idx = idx[0]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = [i for i in sim_scores if i[0] < len(data)]
        sim_scores = sim_scores[1:11]  # Top 10 similar movies

        movie_indices = [i[0] for i in sim_scores]
        return data['Name of movie'].iloc[movie_indices]
    else:
        return "Movie not found in the database."

suggestions = get_recommendations('Happiness')
print("\nRecommendations for 'Happiness':\n", suggestions)


Gradient Boosting Classifier Report:
               precision    recall  f1-score   support

                   0.89      0.94      0.91       119
        High       0.97      1.00      0.99       111
         Low       0.66      0.67      0.67       129
    Moderate       0.73      0.65      0.68       119
   Very High       1.00      1.00      1.00       113

    accuracy                           0.85       591
   macro avg       0.85      0.85      0.85       591
weighted avg       0.84      0.85      0.84       591


Recommendations for 'Happiness':
 960             Moneyball
375              Magnolia
289         Mary and Max.
1569        Mary and Max.
993             25th Hour
677       Midnight Cowboy
706     Kramer vs. Kramer
877                  Zulu
979           Dark Waters
983              Rushmore
Name: Name of movie, dtype: object


In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from imblearn.over_sampling import RandomOverSampler



# Data preprocessing: handling missing values, feature engineering, etc.

# Assuming 'data' contains columns: 'Name of movie', 'Director', 'Star', 'Rating_Category'

# Handling missing values in all columns
data = data.fillna('')  # Fill missing values with an empty string

# Combine selected features into a single column
features = ['Name of movie', 'Director', 'Star']
data['combined_features'] = data[features].apply(lambda x: ' '.join(x), axis=1)

# Create TF-IDF matrix
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['combined_features'])

# Define the target column
target = 'Rating_Category'  # Assuming 'Rating_Category' is the column for movie rating categories

# Handling class imbalance using RandomOverSampler
oversampler = RandomOverSampler(random_state=42)
tfidf_matrix_res, target_res = oversampler.fit_resample(tfidf_matrix, data[target])

# Split resampled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix_res, target_res, test_size=0.2, random_state=42)

# Initialize and train a Decision Tree Classifier
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

# Make predictions using Decision Tree Classifier
dt_predictions = dt_model.predict(X_test)

# Generate a classification report for Decision Tree Classifier
dt_report = classification_report(y_test, dt_predictions)
print("Decision Tree Classifier Report:\n", dt_report)

# Example: Get movie recommendations for a given movie title
def get_recommendations(movie_title, cosine_sim=linear_kernel(tfidf_matrix_res, tfidf_matrix_res), data=data):
    # Implement the recommendation function as mentioned in the previous examples
    pass

suggestions = get_recommendations('Happiness')
print("\nRecommendations for 'Happiness':\n", suggestions)


Decision Tree Classifier Report:
               precision    recall  f1-score   support

                   0.88      1.00      0.93       119
        High       0.97      1.00      0.99       111
         Low       0.75      0.65      0.70       129
    Moderate       0.76      0.74      0.75       119
   Very High       1.00      1.00      1.00       113

    accuracy                           0.87       591
   macro avg       0.87      0.88      0.87       591
weighted avg       0.87      0.87      0.87       591


Recommendations for 'Happiness':
 None


In [ ]:
##################_______________________##################____________###############_________#________###############

In [17]:
print("MOVIES DAATA DONE")

MOVIES DAATA DONE


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat_ws
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Assuming 'data' is a DataFrame in PySpark with columns: 'Name of movie', 'Director', 'Star', 'Rating_Category'
# SparkSession creation
spark = SparkSession.builder.appName("MovieRecommendation").getOrCreate()

# Handling missing values by replacing nulls with empty strings
data = data.fillna('')

# Combining selected features into a single column
data = data.withColumn('combined_features', concat_ws(' ', col('Name of movie'), col('Director'), col('Star')))

# Tokenizing and removing stopwords
tokenizer = Tokenizer(inputCol="combined_features", outputCol="tokens")
data = tokenizer.transform(data)

remover = StopWordsRemover(inputCol="tokens", outputCol="filtered_tokens")
data = remover.transform(data)

# TF-IDF transformation
hashingTF = HashingTF(inputCol="filtered_tokens", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(data)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
data = idfModel.transform(featurizedData)

# Splitting data into training and testing sets
(trainingData, testData) = data.randomSplit([0.8, 0.2], seed=42)

# Logistic Regression model initialization and training
lr = LogisticRegression(labelCol="Rating_Category", featuresCol="features", maxIter=1000)
lrModel = lr.fit(trainingData)

# Making predictions
predictions = lrModel.transform(testData)

# Generating a classification report
evaluator = MulticlassClassificationEvaluator(labelCol="Rating_Category", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)

# Function to get movie suggestions based on movie title
def get_recommendations(movie_title):
    movie = data.filter(col("Name of movie").like(f"%{movie_title}%")).select("Name of movie").take(1)
    if movie:
        similar_movies = lrModel.stages[-1].findSimilarItems(int(movie[0][0]), 10)
        similar_movie_indices = [row[0] for row in similar_movies]
        return data.filter(col("Name of movie").isin(similar_movie_indices)).select("Name of movie").rdd.flatMap(lambda x: x).collect()
    else:
        return "Movie not found in the database."

# Example: Get movie recommendations for a given movie title
suggestions = get_recommendations('Happiness')
print("\nRecommendations for 'Happiness':\n", suggestions)

# Stop the SparkSession
spark.stop()


FileNotFoundError: [WinError 2] The system cannot find the file specified